# Introduction to Keras using the Breast Cancer Wisconsin (Diagnostic) Data Set
*Note: The dataset can be found [here](https://archive.ics.uci.edu/ml/datasets/Breast+Cancer+Wisconsin+%28Diagnostic%29)*

## Project Outline

1. Dataset description
2. Load in data from the file _breast-cancer-wisconsin.data
3. Prune rows from dataset with errors
4. Format Dataset for use in Keras
5. Define network architecture
6. Train network on Data 
7. Score NN Model

## 1. Dataset description

The data located in the file _breast-cancer-wisconsin.data_ is organized as follows: 

```
#  Attribute                     Domain
   -- -----------------------------------------
   1. Sample code number            id number
   2. Clump Thickness               1 - 10
   3. Uniformity of Cell Size       1 - 10
   4. Uniformity of Cell Shape      1 - 10
   5. Marginal Adhesion             1 - 10
   6. Single Epithelial Cell Size   1 - 10
   7. Bare Nuclei                   1 - 10
   8. Bland Chromatin               1 - 10
   9. Normal Nucleoli               1 - 10
  10. Mitoses                       1 - 10
  11. Class:                        (2 for benign, 4 for malignant)
```
More information on this dataset can be found in the _breast-cancer-wisconsin.names_ file.



## 2. Load in data from the file _breast-cancer-wisconsin.data_



In [127]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
# This is for plotting later in the tutorial
%matplotlib inline 


# Read in out csv data
data = pd.read_csv('breast-cancer-wisconsin.data', header=None)

## 3. Prune rows from dataset with errors



In [133]:
# 16 of the data entries have '?' values. We prune them out here. 
data = data[(data.values != '?').all(axis=1)]
data


,0,1,2,3,4,5,6,7,8,9,10
0,1000025,5,1,1,1,2,1,3,1,1,2
1,1002945,5,4,4,5,7,10,3,2,1,2
2,1015425,3,1,1,1,2,2,3,1,1,2
3,1016277,6,8,8,1,3,4,3,7,1,2
4,1017023,4,1,1,3,2,1,3,1,1,2
...,...,...,...,...,...,...,...,...,...,...,...
694,776715,3,1,1,1,3,2,1,1,1,2
695,841769,2,1,1,1,2,1,1,1,1,2
696,888820,5,10,10,3,7,3,8,10,2,4
697,897471,4,8,6,4,3,4,10,6,1,4


## 4. Format Dataset for use in Keras


In [106]:
from keras.utils import np_utils
from sklearn.model_selection import train_test_split

# Get only the last column as our predictions
y = data.iloc[:,-1]
y = y.to_numpy()
y = y.astype('float32')
y /= 2.0
y = y - 1 # make output either zero or one 
#np_y = np_utils.to_categorical(y, 10)


# Get everything but the first and last column as our input data
x = data.iloc[:,1:-1]
x = x.to_numpy()
x = x.astype('float32')
x /= 10.0 # normalize data

x_train, x_test, y_train, y_test = train_test_split(x, y, shuffle=True, test_size=0.1)
y_train = np_utils.to_categorical(y_train, 2)
y_test = np_utils.to_categorical(y_test, 2)


## 5. Define network architecture


In this part of the tutorial you have a large amount of freedome with how you choose to organize your neural network. Here I chose to have simple layers of 16->32->16->2 because I was getting good results with this part. I did try adding some extra layers and making the network wider, but this did not do much to increase the accuracy. 

In [123]:
from keras.models import Sequential
from keras.layers.core import Dense, Activation, Dropout
from keras.optimizers import SGD
np.random.seed(1671) # for reproducibility”

# 10 outputs
# final stage is softmax
model = Sequential()
model.add(Dense(16, input_shape=(9,)))
model.add(Activation('relu'))
model.add(Dense(32))
model.add(Activation('relu'))
model.add(Dense(16))
model.add(Activation('relu'))
model.add(Dense(2))
model.add(Activation('softmax'))
model.summary()

model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['categorical_accuracy'])



Model: "sequential_37"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_125 (Dense)            (None, 16)                160       
_________________________________________________________________
activation_64 (Activation)   (None, 16)                0         
_________________________________________________________________
dense_126 (Dense)            (None, 32)                544       
_________________________________________________________________
activation_65 (Activation)   (None, 32)                0         
_________________________________________________________________
dense_127 (Dense)            (None, 16)                528       
_________________________________________________________________
activation_66 (Activation)   (None, 16)                0         
_________________________________________________________________
dense_128 (Dense)            (None, 2)               

## 6. Train network on Data 

In this section we also have the opprotunity to custimize our hyperparameters quite a bit. I tried a couple of diffrent batch sizes and epochs to check which one resulted in the best accuracy. If I wanted to be very particular and pick only the very best hyperparams, then I would loop though multiple training cycles and try a multitude of batch sizes and epochs. 


In [124]:
    history = model.fit(x_train, y_train, batch_size=16, epochs=200, verbose=1, validation_split=0.2)

Train on 491 samples, validate on 123 samples
Epoch 1/200
491/491 [==============================] - 0s 374us/step - loss: 0.6847 - categorical_accuracy: 0.6436 - val_loss: 0.6509 - val_categorical_accuracy: 0.6585
Epoch 2/200
491/491 [==============================] - 0s 95us/step - loss: 0.6114 - categorical_accuracy: 0.6436 - val_loss: 0.5638 - val_categorical_accuracy: 0.6667
Epoch 3/200
491/491 [==============================] - 0s 93us/step - loss: 0.4884 - categorical_accuracy: 0.8391 - val_loss: 0.4189 - val_categorical_accuracy: 0.8862
Epoch 4/200
491/491 [==============================] - 0s 87us/step - loss: 0.3073 - categorical_accuracy: 0.9532 - val_loss: 0.2568 - val_categorical_accuracy: 0.9187
Epoch 5/200
491/491 [==============================] - 0s 81us/step - loss: 0.1569 - categorical_accuracy: 0.9654 - val_loss: 0.1718 - val_categorical_accuracy: 0.9187
Epoch 6/200
491/491 [==============================] - 0s 96us/step - loss: 0.1023 - categorical_accuracy: 0.9695

## 7. Score NN Model


In [125]:
score = model.evaluate(x_test, y_test, verbose=1)
print("Test score:", score[0])
print('Test accuracy:', score[1])

69/69 [==============================] - 0s 43us/step
Test score: 0.21961637424386066
Test accuracy: 0.9855072498321533
